# File for the models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler, QuantileTransformer, \
    PowerTransformer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
import lightgbm as lgbm

import xgboost as xgb
from sklearn.linear_model import Lasso,LinearRegression,Ridge
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
df_train = pd.read_csv("data/training_data_preprocessed.csv")


### LGBM model

In [2]:
# cols = df_train.columns[df_train].tolist()

    # cols_missing = df_comb.columns[df_comb.isnull().any()].tolist()
columns_list = []
for col in df_train.columns:
    if col != 'prop_id':
    # print(col)
        columns_list.append(col)
print(columns_list)

NameError: name 'df_train' is not defined

In [3]:
# train, test = train_test_split(dataset, test_size=0.2, random_state=25)

# features_train = ['activity', 'appCat.values', 'call', 'circumplex.values', 'screen', 'sms']

features_train = []
for col in df_train.columns:
    if col != 'prop_id':
    # print(col)
        features_train.append(col)
print(features_train)
target_train = 'prop_id'

X_train, X_val, y_train, y_val = train_test_split(
    df_train[features_train], df_train[target_train], test_size=0.2)

train_data = lgbm.Dataset(X_train, label=y_train)

val_data = lgbm.Dataset(X_val, label=y_val)

NameError: name 'df_train' is not defined

In [4]:
# determine style
sns.set_theme(style="darkgrid")

#basic parameter:
parameters = {
    # 'application': 'binary',
    # 'objective': 'binary',
    'metric': 'mse',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 25,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.05,
    'verbose': 0
}

# determine model
model = lgbm.train(parameters,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)

# plot and show feature importance
ax = lgbm.plot_importance(model, max_num_features=40, figsize=(15, 15))
plt.show()

NameError: name 'sns' is not defined

In [5]:
hyper_params = {
    'random_state': 420,
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['mse'],
    'learning_rate': 0.1,
    "num_leaves": 256,
    'max_depth': 9,
    # 'min_data_in_leaf': 70,
    # "max_bin": 512,
}

train_data = lgbm.Dataset(X_train, label=y_train, categorical_feature='auto')
val_data = lgbm.Dataset(X_val, label=y_val, categorical_feature='auto')

model = lgbm.train(hyper_params, train_data, num_boost_round=500, verbose_eval=False)

NameError: name 'X_train' is not defined

In [6]:
y_pred = model.predict(X_val[features_train])

predictions = pd.DataFrame({'predicted': y_pred, 'actual': y_val})

NameError: name 'model' is not defined

In [7]:
mse_lgbm = mean_squared_error(predictions['actual'], predictions['predicted'])
mae_lgbm = mean_absolute_error(predictions['actual'], predictions['predicted'])
rmse_lgbm = np.sqrt(mean_squared_error(predictions['actual'], predictions['predicted']))

print(f"MSE is: {mse_lgbm}")
print(f"MAE is: {mae_lgbm}")
print(f"RMSE is: {rmse_lgbm}")

NameError: name 'mean_squared_error' is not defined

In [8]:
par_xgbc = {
            #   'tree_method': 'gpu_hist', 
              'objective':'multi:softmax',
              'num_class': 100,
              'learning_rate': 0.08,
              'max_depth': 8,
              'min_child_weight': 9,
              'silent': 1,
              'subsample': 0.8,
              'colsample_bytree': 0.7,
              'n_estimators': 600,
              'seed': 42
}

XGBC = xgb.XGBClassifier(**par_xgbc)
Las = Lasso()
LinR = LinearRegression()
Rid = Ridge()
Rfc = RandomForestRegressor(random_state=2)
Dtc = DecisionTreeRegressor(random_state = 2)
Boost_Lin = AdaBoostRegressor(base_estimator=LinR,random_state=2)
Boost_las = AdaBoostRegressor(base_estimator=Las,random_state=2)
Boost_rid = AdaBoostRegressor(base_estimator=Rid,random_state=2)
Bg_Lin = BaggingRegressor(base_estimator=LinR,random_state=2)
Bg_las = BaggingRegressor(base_estimator=Las,random_state=2)
Bg_rid = BaggingRegressor(base_estimator=Rid,random_state=2)

In [9]:
origin_features = []
for col in df_train.columns:
    if col != 'prop_id':
    # print(col)
        origin_features.append(col)
print(origin_features)
target_train = 'prop_id'

# using the full set for training
training_set = df_train[origin_features + [target_train]].sample(10000)

# separating the variable that is to be predicted by the model from the rest
X = training_set.drop(target_train, axis=1)
y = training_set[target_train]

features = X.columns

# built a validation set on flights_train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

NameError: name 'df_train' is not defined

In [10]:
model_columns = []
models_compare = pd.DataFrame(columns = model_columns)
results=[]

row_index = 0
for model, name in zip([XGBC, Las,LinR,Rid,Dtc,Rfc,Boost_Lin,Boost_las,Boost_rid,Bg_Lin,Bg_las,Bg_rid], 
     ['XGBC','Lasso','Linear Regression','Ridge','Random forest Regressor','Decision Tree Regressor','Boosted Linear',
      'Boosted Lasso','Boosted Ridge','Bagged Linear','Bagged Lasso','Bagged Ridge']):

    model1 = model.fit(X_train,y_train)
    Y_predict = model1.predict(X_val)
    results.append(Y_predict)
    mae_score = mean_absolute_error(y_val, Y_predict)
    mse_score = mean_squared_error(y_val, Y_predict)
    rmse_score = np.sqrt(mean_squared_error(y_val, Y_predict))
    models_compare.loc[row_index,'Model Name'] = name
    models_compare.loc[row_index, 'MAE'] = mae_score
    models_compare.loc[row_index, 'MSE'] = mse_score
    models_compare.loc[row_index, 'RMSE'] = rmse_score   
    row_index+=1

models_compare.sort_values(by = ['MSE'], ascending = True, inplace = True)    
models_compare

NameError: name 'pd' is not defined

In [11]:
plt.subplots(figsize=(15,4))
sns.lineplot(x="Model Name", y="MSE",data=models_compare,label='Val Accuracy')
# sns.lineplot(x="Model Name", y="Model Test Accuracy",data=models_compare,label='Test Accuracy')
plt.xticks(rotation=90)
plt.title('Model Accuracy Comparison')
plt.legend()
plt.savefig('models/output_models.png')
plt.show()

NameError: name 'plt' is not defined